# Regular Expressions

Regular Expressions (REs) are a flexible way to define textual patterns. Defining REs in Python is straightforward:

In [30]:
import re

pattern = re.compile('[bcrh]at')
pattern2 = re.compile('(.*)([bcrh]at)(.*)')

We can then use the pattern to `search()` or `match()` strings to it. 

`search()` will return a result if the pattern occurs **anywhere** in the input string.

`match()` will only return a result if the pattern **completely** matches the input string.

In [46]:
word = 'the batter won the game in a hat'
matches = re.findall(pattern2, word) # won't return a a result, i.e., matches = None
searches = re.search(pattern2, word) # finds a substring

In [47]:
print(matches)
print(searches)
# print(matches.groups())

[('the batter won the game in a ', 'hat', '')]
<_sre.SRE_Match object; span=(0, 32), match='the batter won the game in a hat'>


Both have a number of attributes to access the results. 
- `span()` gives us a tuple of the substring that matches
- `group()`returns the matched substring

In [48]:
span = searches.span()
word[span[0]:span[1]], span

('the batter won the game in a hat', (0, 32))

In [49]:
searches.groups()

('the batter won the game in a ', 'hat', '')

If we have used several RE groups (in brackets `()`), we can access them individually via `groups()`

In [59]:
word = 'preconstitutionalism'
affixes = re.compile('(...).+(...)')
re.search(affixes, word).groups()

('pre', 'ism')

For the email address finder, we can use a more advanced pattern and test it:

In [64]:
email = re.compile('^[A-Za-z0-9][A-Za-z0-9\.-]*@[A-Za-z0-9][A-Za-z0-9\.-]+\.[A-Za-z0-9\.-][A-Za-z0-9\.-][A-Za-z0-9\.-]?$')
# for address in ['me.@unibocconi.it', '@web.de', '.@gmx.com', 'not working@aol.com']:

for address in 'notMyFault@webmail.gov.uk,smithie123@gmx,Free stuff@unibocconi.it,mark_my_words@hotmail;com,truthOrDare@webmail.in,look@me@twitter.com,how2GetAnts@aol.dfdsfgfdsgfd'.split(','):
    print(address, re.match(email, address))
# bocconi_address = re.match(email, 'me.@unibocconi.it')

notMyFault@webmail.gov.uk <_sre.SRE_Match object; span=(0, 25), match='notMyFault@webmail.gov.uk'>
smithie123@gmx None
Free stuff@unibocconi.it None
mark_my_words@hotmail;com None
truthOrDare@webmail.in <_sre.SRE_Match object; span=(0, 22), match='truthOrDare@webmail.in'>
look@me@twitter.com None
how2GetAnts@aol.dfdsfgfdsgfd None


We can also use the pattern to replace elements of a string that match with `sub()`

In [67]:
print('Are you all awake???'.replace('???', '!'))

numbers = re.compile('[0-9]')
number_sentence = 'Back in the 90s, when I was a 12-year-old, a CD cost just 15,99EUR!'
re.sub(numbers, '0', number_sentence)

Are you all awake!


'Back in the 00s, when I was a 00-year-old, a CD cost just 00,00EUR!'

## Exercise

Write a RegEx to remove all user names from the tweets and replace them with the token "@USER"

In [70]:
tweets = [line.strip() for line in open('../data/tweets.txt', encoding='utf8')]
cleaned_tweets = []
# your code here
to_replace = re.compile('@[A-Za-z0-9_]+')

for d, sentence in enumerate(tweets):
    corrected = re.sub(to_replace, '@USER', sentence)
#     print(corrected)
    cleaned_tweets.append(corrected)
print(cleaned_tweets[:10])

['@USER I think a lot of people just enjoy being a pain in the ass on there', 'Best get ready sunbed and dinner with nana today :)', '@USER thats awesome!', 'Loving this weather', '“@USER: Just seen an absolute idiot in shorts! Be serious!” Desperado gentleman', '@USER trying to resist a hardcore rave haha! Resisting towns a doddle! Posh dance floor should wear them in quite easy xx', '59 days until @USER!!! Wooo @USER #cannotwait', 'That was the dumbest tweet i ever seen', 'Oh what to do on this fine sunny day?', '@USER hows the fish ? Hope they r ok. Xx']


## Exercise

Write a RegEx to search for all hashtags containing the word `good` in them.

In [ ]:
# your code here
re.findall

# Linguistic Analysis

Let's look at the different levels of linguistic analysis. First, we need some text. Let's take *Moby Dick* from Project Gutenberg, as a list of strings:

In [71]:
documents = [line.strip() 
             for line in open('../data/moby_dick.txt', 
                              encoding='utf8').readlines()]
print(documents[1])

Call me Ishmael .


We will use the `spacy` library for a lot of the analyses. Here, we load it:

In [72]:
import spacy

nlp = spacy.load('en')

### Usage:

We can now call `nlp()` as a function on any text. By default, it will perform a number of analyses:
- tokenization
- sentence splitting
- lemmatization
- part of speech tagging
- dependency parsing
- named entity recognition

To speed up analysis, we can disable some of these analyses if we do not need it:
```
nlp = spacy.load('en', disable=['tokenizer', 'tagger', 'parser', 'ner'])
```


The result is an iterator over the sentences (if called on a text), or tokens (if called on a sentence). Each token has a range of properties see [here](https://spacy.io/api/token#attributes). We will use a few of them in the following:

- `text`: the actual word
- `lemma_`: the dictionary entry of a word
- `pos_`: the part of speech
- `dep`: dependency relation
- `is_punct`: check whether word is punctuation
- `is_stop`: check whether word is a stop word

## Tokenization
Before we do anything, we need to insert spaces into the data.

In [74]:
tokens = [[token.text for token in nlp(sentence)] 
          for sentence in documents[:100]]
tokens

[token.text for token in nlp('Here, without further ado, is another sentence!')]

['Here',
 ',',
 'without',
 'further',
 'ado',
 ',',
 'is',
 'another',
 'sentence',
 '!']

### Exercise

Collect counts over the tokens. What is the most frequent token?

In [75]:
# your code here
from collections import Counter

c = Counter([token for sentence in tokens for token in sentence])
# c.most_common(1)
c.most_common(10)

[(',', 157),
 ('the', 111),
 ('.', 78),
 ('of', 75),
 ('a', 67),
 ('and', 61),
 ('to', 49),
 ('in', 43),
 ('I', 41),
 ('is', 32)]

## Lemmatization
We want to get the dictionary form of each word, to reduce variation.

In [78]:
lemmas = [token.lemma_ 
          for sentence in documents[:100] 
          for token in nlp(sentence)]
lemmas

['looming',
 '.',
 'call',
 '-PRON-',
 'ishmael',
 '.',
 'some',
 'year',
 'ago',
 '--',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 '--',
 'have',
 'little',
 'or',
 'no',
 'money',
 'in',
 '-PRON-',
 'purse',
 ',',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 '-PRON-',
 'on',
 'shore',
 ',',
 '-PRON-',
 'think',
 '-PRON-',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part',
 'of',
 'the',
 'world',
 '.',
 '-PRON-',
 'be',
 'a',
 'way',
 '-PRON-',
 'have',
 'of',
 'drive',
 'off',
 'the',
 'spleen',
 'and',
 'regulate',
 'the',
 'circulation',
 '.',
 'whenever',
 '-PRON-',
 'find',
 '-PRON-',
 'grow',
 'grim',
 'about',
 'the',
 'mouth',
 ';',
 'whenever',
 '-PRON-',
 'be',
 'a',
 'damp',
 ',',
 'drizzly',
 'november',
 'in',
 '-PRON-',
 'soul',
 ';',
 'whenever',
 '-PRON-',
 'find',
 '-PRON-',
 'involuntarily',
 'pause',
 'before',
 'coffin',
 'warehouse',
 ',',
 'and',
 'bring',
 'up',
 'the',
 'rear',
 'of',
 'every',
 'funera

### Exercise
Right now, the lemmas of all pronouns are collapsed into `-PRON-`. Change the code to preserve the original word (as lower case).

In [84]:
# your code here
lemmas_with_pron = [token.lemma_ 
                    if token.lemma_ != '-PRON-' else token.lower_ 
                    for sentence in documents[:10] 
                    for token in nlp(sentence)]
lemmas_with_pron

['looming',
 '.',
 'call',
 'me',
 'ishmael',
 '.',
 'some',
 'year',
 'ago',
 '--',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 '--',
 'have',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 ',',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 ',',
 'i',
 'think',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part',
 'of',
 'the',
 'world',
 '.',
 'it',
 'be',
 'a',
 'way',
 'i',
 'have',
 'of',
 'drive',
 'off',
 'the',
 'spleen',
 'and',
 'regulate',
 'the',
 'circulation',
 '.',
 'whenever',
 'i',
 'find',
 'myself',
 'grow',
 'grim',
 'about',
 'the',
 'mouth',
 ';',
 'whenever',
 'it',
 'be',
 'a',
 'damp',
 ',',
 'drizzly',
 'november',
 'in',
 'my',
 'soul',
 ';',
 'whenever',
 'i',
 'find',
 'myself',
 'involuntarily',
 'pause',
 'before',
 'coffin',
 'warehouse',
 ',',
 'and',
 'bring',
 'up',
 'the',
 'rear',
 'of',
 'every',
 'funeral',
 'i',
 'meet',
 ';',
 'and',
 'especially',
 

## Stemming
A more aggressive way of removing variation is *stemming*.

In [85]:
from nltk import SnowballStemmer

stemmer = SnowballStemmer('english')
stems = [[stemmer.stem(token) 
          for token in sentence] 
         for sentence in tokens]
stems

[['loom', '.'],
 ['call', 'me', 'ishmael', '.'],
 ['some',
  'year',
  'ago',
  '--',
  'never',
  'mind',
  'how',
  'long',
  'precis',
  '--',
  'have',
  'littl',
  'or',
  'no',
  'money',
  'in',
  'my',
  'purs',
  ',',
  'and',
  'noth',
  'particular',
  'to',
  'interest',
  'me',
  'on',
  'shore',
  ',',
  'i',
  'thought',
  'i',
  'would',
  'sail',
  'about',
  'a',
  'littl',
  'and',
  'see',
  'the',
  'wateri',
  'part',
  'of',
  'the',
  'world',
  '.'],
 ['it',
  'is',
  'a',
  'way',
  'i',
  'have',
  'of',
  'drive',
  'off',
  'the',
  'spleen',
  'and',
  'regul',
  'the',
  'circul',
  '.'],
 ['whenev',
  'i',
  'find',
  'myself',
  'grow',
  'grim',
  'about',
  'the',
  'mouth',
  ';',
  'whenev',
  'it',
  'is',
  'a',
  'damp',
  ',',
  'drizzli',
  'novemb',
  'in',
  'my',
  'soul',
  ';',
  'whenev',
  'i',
  'find',
  'myself',
  'involuntarili',
  'paus',
  'befor',
  'coffin',
  'warehous',
  ',',
  'and',
  'bring',
  'up',
  'the',
  'rear',
  '

### Exercise

Keep track of the most frequent word for each stem in `tokens`. Hint: use a nested `defaultdict`.

- How many word forms does the stem `hand` have?
- What is the most common word form for the stems `respect` and `whale`? What happened there?

In [86]:
# your code here
from collections import defaultdict
stem_origins = defaultdict(lambda: defaultdict(int))

for sentence in tokens:
    for token in sentence:
        stem = stemmer.stem(token)
        stem_origins[stem][token.lower()] += 1
stem_origins['whale']

defaultdict(int, {'whaling': 4, 'whale': 2})

## Parts of speech
We can extract the part of speech for every word with the `pos_` atttribute.

In [87]:
pos = [[token.pos_ 
        for token in nlp(sentence)] 
       for sentence in documents[:100]]
pos

[['NOUN', 'PUNCT'],
 ['VERB', 'PRON', 'PROPN', 'PUNCT'],
 ['DET',
  'NOUN',
  'ADV',
  'PUNCT',
  'ADV',
  'VERB',
  'ADV',
  'ADV',
  'ADV',
  'PUNCT',
  'VERB',
  'ADJ',
  'CCONJ',
  'DET',
  'NOUN',
  'ADP',
  'ADJ',
  'NOUN',
  'PUNCT',
  'CCONJ',
  'NOUN',
  'ADJ',
  'ADP',
  'VERB',
  'PRON',
  'ADP',
  'NOUN',
  'PUNCT',
  'PRON',
  'VERB',
  'PRON',
  'VERB',
  'VERB',
  'ADP',
  'DET',
  'ADJ',
  'CCONJ',
  'VERB',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'PUNCT'],
 ['PRON',
  'VERB',
  'DET',
  'NOUN',
  'PRON',
  'VERB',
  'ADP',
  'VERB',
  'PART',
  'DET',
  'NOUN',
  'CCONJ',
  'VERB',
  'DET',
  'NOUN',
  'PUNCT'],
 ['ADV',
  'PRON',
  'VERB',
  'PRON',
  'VERB',
  'ADJ',
  'ADP',
  'DET',
  'NOUN',
  'PUNCT',
  'ADV',
  'PRON',
  'VERB',
  'DET',
  'NOUN',
  'PUNCT',
  'VERB',
  'PROPN',
  'ADP',
  'ADJ',
  'NOUN',
  'PUNCT',
  'ADV',
  'PRON',
  'VERB',
  'PRON',
  'ADV',
  'VERB',
  'ADP',
  'NOUN',
  'NOUN',
  'PUNCT',
  'CCONJ',
  'VERB',
  'PART',


### Exercise
Print out the words in the first 10 sentences, but remove all words that are not nouns, verbs, adjectives, adverbs, or proper names.

In [ ]:
# your code here
content_words = [[token.lemma_ for token in nlp(sentence) if token.pos_ in {'NOUN', 'VERB', 'ADJ', 'ADV', 'PROPN'}] for sentence in documents[:10]]
content_words


## Named Entities
For each noun phrase, we can infer the semantic type of it.

In [89]:
entities = [[(entity.text, entity.label_) 
             for entity in nlp(sentence).ents]
            for sentence in documents[:50]]
entities
nlp('John gave a book to Mary and Celia in Cardiff').ents

(John, Mary and Celia, Cardiff)

### Exercise
Who are the 5 most frequently named people in the first 500 sentence?

In [ ]:
# your code here
top_5_entities = [entity.text 
                  for sentence in documents[:500]
                  for entity in nlp(sentence).ents
                  if entity.label_ == 'PERSON'
]

Counter(top_5_entities).most_common(5)

### Exercise

Use the text below to extract all entities. 
- Create tuples of `(lemma, NER type)`
- Collect counts over the tuples
- Look at the 10 most frequent tuples: how many of them are wrong? Why? Discuss with a neighbor.


In [ ]:
text = """
Seville.
Summers in the flamboyant Andalucían capital often nudge 40C, but spring is a delight, with the parks in bloom and the scent of orange blossom and jasmine in the air. And in Semana Santa (Holy Week, 14-20 April) the streets come alive with floats and processions. There is also the raucous annual Feria de Abril – a week-long fiesta of parades, flamenco and partying long into the night (4-11 May; expect higher hotel prices if you visit then).
Seville is a romantic and energetic place, with sights aplenty, from the Unesco-listed cathedral – the largest Gothic cathedral in the world – to the beautiful Alcázar royal palace. But days here are best spent simply wandering the medieval streets of Santa Cruz and along the river to La Real Maestranza, Spain’s most spectacular bullring.
Seville is the birthplace of tapas and perfect for a foodie break – join a tapas tour (try devoursevillefoodtours.com), or stop at the countless bars for a glass of sherry with local jamón ibérico (check out Bar Las Teresas in Santa Cruz or historic Casa Morales in Constitución). Great food markets include the Feria, the oldest, and the wooden, futuristic-looking Metropol Parasol.
Nightlife is, unsurprisingly, late and lively. For flamenco, try one of the peñas, or flamenco social clubs – Torres Macarena on C/Torrijano, perhaps – with bars open across town until the early hours.
Book it: In an atmospheric 18th-century house, the Hospes Casa del Rey de Baeza is a lovely place to stay in lively Santa Cruz. Doubles from £133 room only, hospes.com
Trieste.
By April, temperatures are on the rise in Trieste and in the late 20s by May. It is far less touristy than the likes of Florence or Rome, and spring sees the city’s lovely restaurants and bars populated almost exclusively by locals.
A city with a proud coffee-drinking culture – Illy has its headquarters here – Trieste has many venerable cafes, including the dazzling mirror-walled Caffè degli Specchi on the Piazza Unità d’Italia – said to be Europe’s biggest seaside piazza – and the elegant Caffè San Marco, which has a good bookshop. James Joyce was a regular when he lived here between 1904-1915. You can learn all about him at the excellent museum, which also has a free, downloadable themed walk on its website (museojoycetrieste.it).
Above Trieste is a vast limestone plateau known as the carso (or karst). Travel up to Villa Opicina on the edge of the region by bus.
There are several trattorie, but for a real treat catch a cab to one of the 30 or so osmize – farm restaurants that sell their wines, cured meats, cheese, honey, fruit and veg; traditionally, they were open eight, 16 or 24 days per year (“osmi” means “eighth” in Slovene) but this now varies – check the app at osmize.com for details.
Book it: Stay at the palatial, seafront Savoia Excelsior Palace, Jan Morris’s pad in her book Trieste and the Meaning of Nowhere. Doubles from £127 room only, starhotelscollezione.com
Belgrade.
As Belgrade shrugs off the snow, cafe tables start colonising the pavements again. Not that Serbia’s capital hibernates during the winter, but spring brings a freshness worth savouring before summer’s 40C heat kicks in.
You feel it especially in Kalemegdan, the huge park and fortress hugging the confluence of the Danube and Sava rivers. Down below are wide riverside paths that offer superb cycling all the way to the attractive suburb of Zemun. Follow the Sava southwards to reach the river island of Ada Ciganlija – open all year round for walks and bike rides and usually warm enough in May for a swim.
Although barely a month goes by without a festival, the pace picks up during spring. The Belgrade Dance Festival attracts dance companies from around the world. Classical guitar gets its own spotlight during the Guitar Art Festivalfrom 12-17 March, and from 26-28 April you can join the Orthodox Easter festivities. The landmark Mikser House in Savamala may have closed, but its Mikser festival still celebrates the best in Balkan design (24-26 May).
With the long-awaited reopening of the National Museum of Serbia and the Museum of Contemporary Art, Belgrade has some cultural heavyweights to add to its dizzyingly varied restaurant scene. Head to the Dorćol district for cheap cocktails in Blaznavac’s psychedelic garden before a Balkan-Mediterranean dinner in cosy Tezga (Strahinjića Bana 82, on Facebook).
Book it: Set in a handsome 1929 villa in Dorćol, Smokvica has six stylish rooms as well as a restaurant with a courtyard garden. Doubles from €70 room only, smokvica.rs
Montpellier.
Montpellier combines easy elegance and a vibrant cultural scene with a youthful buzz – its university, founded in the 13th century, counts radical satirist Rabelais among its alumni and some 60,000 students live here.
The medieval centre is a maze made for wandering, with 16 leafy squares – in spring, all green and abuzz with alfresco cafe life. The vast, pedestrianised Place de la Comédie connects the old town with the striking new Antigone district, replete with modern, neoclassical-style buildings.
For a fine-art foray, head to Musée Fabre – one of the biggest in France – or nearby photography museum Pavillon Populaire. Montpellier boasts the oldest botanical garden in France, too, dating from 1593 and particularly beautiful in April and May. Independent boutiques, opera houses, markets laden with Languedoc produce (look for oysters and asparagus) and great dining options add to the appeal. Le Petit Jardin is a bistro and lovely garden restaurant near the old cathedral.
Beyond the city, discover vineyards like Mas de Daumas Gassac or hike up Pic St-Loup in the Cévennes foothills for views over the coast. The beach and charming seaside town of Palavas-les-Flots is just 10km away – a tram ride or easy cycle (rent bicycles from Ville et Vélo).
Book it: Hotel Le Guilhem is a 16th-century building in the historic centre with cathedral views from its terrace (where breakfast is served, weather permitting). Doubles from £83 room only, leguilhem.com
Berlin.
After a long, cold winter, Berliners waste no time in celebrating the return of the sun, with beer gardens and flea markets reopening all over the city. Try picturesque Cafe Am Neuen Seefor lakeside pizza and beer in the Tiergarten, and hang out with the hipsters along the canal at Berlin’s coolest flohmarkt, surrounded by cherry blossom trees, a charming place to browse and have a couple of beers.
For the best blossom, though, head to one of the trails along the line of the wall, the Mauerweg, near S-bahn Bornholmer Strasse, or Lichterfelde Süd, where the trees were gifted by the Japanese to celebrate German reunification in 1989.
Berlin is set for two big anniversaries this year: it’s 100 years since the Bauhaus was founded and 30 years since the wall came down, so check out events including exhibitions and dances themed around the former (see visitberlin.de for more information) or get a feel for the creative chaos of post-wall Berlin at the multimedia Nineties Berlin exhibition.
Boat tours along the river and canals are superb in spring but, generally, cycling is the best way to get around (many hotels and hostels have their own). And if urban life gets a bit much, do as the Berliners do and head out to one of the many city lakes, such as Schlachtensee or Wannsee, directly accessible by U and S-bahn.
Book it: Boutique Hotel Oderberger in a trendy, central location has doubles from £113, hotel-oderberger.berlin, or try indoor caravanning at Hüttenpalast, doubles from £61 room only, huettenpalast.de
"""

# your code here

## Parsing
For each word, we can extract the word it is grammatically related to, plus the type of the relation.

In [93]:
[[(c.text, c.dep_, c.head.text) for c in nlp(sentence)] 
 for sentence in ['We measured the results.', 'The results were measured by us.']]

[[('We', 'nsubj', 'measured'),
  ('measured', 'ROOT', 'measured'),
  ('the', 'det', 'results'),
  ('results', 'dobj', 'measured'),
  ('.', 'punct', 'measured')],
 [('The', 'det', 'results'),
  ('results', 'nsubjpass', 'measured'),
  ('were', 'auxpass', 'measured'),
  ('measured', 'ROOT', 'measured'),
  ('by', 'agent', 'measured'),
  ('us', 'pobj', 'by'),
  ('.', 'punct', 'measured')]]

Instead of doing this at a word-by-word basis, we can do it by larger chunks, the noun phrases.

In [ ]:
[[(c.text, c.root.head.text, c.root.dep_) for c in nlp(sentence).noun_chunks] 
 for sentence in documents[:100]]

## Exercise
How does Melville describe nouns? Extract all the pairs related by `amod`.

In [ ]:
# your code here


## Syntactic $n$-grams


In [ ]:
features = [' '.join(["{}_{}".format(c.lemma_, c.head.lemma_) 
                      for c in nlp(sentence)])
            for sentence in documents[:100]]

syntax_vectorizer = CountVectorizer()
X = syntax_vectorizer.fit_transform(features)

In [ ]:
print(syntax_vectorizer.vocabulary_)